<a href="https://colab.research.google.com/github/diegomendesmoreno/job_match/blob/main/Match_de_Vaga_Avalia%C3%A7%C3%A3o_do_CV_sobre_vagas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicializando (dura aproximadamente 20 segundos)

In [1]:
!pip install -q -U google-generativeai

# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

api_key = userdata.get('SECRET_KEY')
GOOGLE_API_KEY=api_key
genai.configure(api_key=GOOGLE_API_KEY)

# Configuração

In [2]:
model_name = "gemini-1.5-pro-latest"

safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

model = genai.GenerativeModel(model_name=model_name,
                              safety_settings=safety_settings)

# Dados para few-shot prompting (fsp)

In [3]:
system_instructions = "- Você é um especialista em recrutamento, seleção e contratação de pessoas. - Você sempre está falando com a pessoa candidata que está procurando uma vaga de trabalho. - Você vai analisar o currículo e experiência da pessoa candidata e a descrição da vaga desejada. - Seu trabalho é dar uma nota de 0 a 10. - 0 significa que a candidata não tem nenhuma qualificação para a vaga e não tem nenhuma chance de ser contratada. - 10 significa que a candidata tem todas as qualificações necessárias para a vaga e tem muita chance de ser contratada."

fsp_sample_cv = "Perfil Sou Engenheiro Eletrônico com MBA em Engenharia de Software e possuo experiência no desenvolvimento de aplicações industriais e web. Minhas principais habilidades estão relacionadas ao desenvolvimento de software em geral, seguindo práticas ágeis e de engenharia de software. Habilidades Programação: C, Python Engenharia de Software: Git, GitHub, Metodologia Ágil, Unity/MinUnit (testes), Linux Idiomas: Português (nativo), Inglês (fluente), Espanhol (avançado), Francês (intermediário) Habilidades Interpessoais: Aprendizado contínuo, comunicação com o cliente Formação MBA em Engenharia de Software - 2022 EMPRESA - São Paulo, SP Engenharia Eletrônica - 2015 EMPRESA - São Bernardo do Campo, SP Engenharia Elétrica - 2013 EMPRESA - Birmingham, AL (EUA) Intercâmbio Acadêmico de um ano pelo Programa Ciências sem Fronteiras, patrocinado pela CAPES Experiência Profissional Engenheiro de Aplicações de Campo - Fevereiro de 2022 - Presente EMPRESA - São Paulo, SP Suporte ao desenvolvimento de software embarcado em C/C++ Suporte em especificidades de arquitetura de microcontroladores (ex: ARM Cortex-M RA e Synergy, RX, RL78, DA14531, DA16200 e RZ) em proteção de memória, modos de inicialização, baixo consumo de energia, memória de configuração de opções, layout de memória, etc. Suporte na utilização de periféricos de microcontroladores como CAN, SPI, I2C, UART, Temporizadores, etc. Teste, solução de problemas, depuração e validação de software e hardware para inicialização de placas e problemas gerais de funcionamento usando ferramentas de bancada (ex: osciloscópio, analisador lógico) e seguindo esquemas de circuito e desenhos de layout de PCB Suporte para uso e configuração de ambientes de desenvolvimento integrado (IDEs) e ferramentas de depuração de software Ministrou sessões de treinamento técnico e demonstrações de produtos em clientes e parceiros Parte de uma equipe remota global com função prática voltada para o cliente Engenheiro de Software Embarcado - Fevereiro de 2021 - Fevereiro de 2022 EMPRESA - São Paulo, SP Desenvolvimento de software embarcado para produtos de plantio de precisão (Agronegócio) seguindo as melhores práticas de desenvolvimento de software Adicionou uma interface API CAN (Controller Area Network) ao nosso medidor de sementes a vácuo para oferecer suporte a um novo tipo de sensor de sementes (Tecnologias: C, CAN) Expandiu a interface API de um gateway de sensor sem fio adicionando novos recursos (Tecnologias: C, CAN) Desenvolveu um projeto de IoT de Prova de Conceito (PoC) ponta a ponta de um gateway de sensor sem fio com conectividade Wi-Fi que monitora sensores por meio de um aplicativo da Web (Tecnologias: C, Wi-Fi, servidor da Web, API RESTful, Javascript, HTML, CSS) Desenvolveu uma aplicação para PC com GUI (Interface Gráfica do Usuário) para monitorar sensores em tempo real, mostrar métricas importantes e gerar relatórios (Tecnologias: Python) Trabalhou para adicionar, refatorar e integrar código em uma grande base de código descentralizada Realizou e foi submetido a revisões frequentes de código junto com a equipe de desenvolvimento Trabalhando com práticas e ferramentas Agile (Jira, Git, BitBucket, Confluence) Implementou uma prática de documentação amigável ao controle de versão com Markdown e PlantUML para documentos de produtos Engenheiro de Aplicações de Campo - Maio de 2014 - Fevereiro de 2021 EMPRESA - São Paulo, SP Desenvolvimento de software embarcado para microcontroladores de 8 bits, 16 bits e 32 bits para diversas aplicações Desenvolvimento C e Python para Linux Embarcado Personalização de Linux Embarcado usando o Projeto Yocto Desenvolvimento de interfaces gráficas de usuário (GUI) com Qt Uso ativo de Git/GitHub para controle de versão Uso de serviços da Web IoT (MQTT/HTTP) como Tago.io e AWS Experiência com pilhas de controle Wi-Fi (IEEE 802.11), Bluetooth Low Energy (BLE) e TFT Treinamento técnico em sistemas embarcados Suporte técnico em sistemas embarcados e eletrônica de potência Desenvolvimento e entrega de demonstrações de produtos e protótipos Visitas técnicas a clientes Especificação de componentes eletrônicos e referência cruzada para várias aplicações"

# Example prompt/answer 1
fsp_sample_job1 = "Descrição da Vaga: Engenheiro(a) de Software (Early Career) Qualificações Mínimas: Graduação em Ciência da Computação, áreas relacionadas ou experiência prática equivalente. 1 ano de experiência em desenvolvimento de software em uma ou mais linguagens de programação (Ex: Python, C, C++, Java, JavaScript). Qualificações Desejáveis: Mestrado ou Doutorado em Ciência da Computação ou áreas relacionadas. Experiência com desenvolvimento de tecnologias acessíveis. Disponibilidade para iniciar em 2024. Sobre a Vaga: Os engenheiros de software da EMPRESA desenvolvem tecnologias de última geração que mudam a forma como bilhões de usuários se conectam, exploram e interagem com a informação e uns com os outros. Nossos produtos precisam lidar com informações em grande escala e se estender muito além da pesquisa na web. Buscamos engenheiros que tragam ideias inovadoras de todas as áreas, incluindo recuperação de informação, computação distribuída, design de sistemas de larga escala, redes e armazenamento de dados, segurança, inteligência artificial, processamento de linguagem natural, design de interface do usuário e desenvolvimento mobile, entre outras. A lista é extensa e cresce a cada dia. Como engenheiro(a) de software, você trabalhará em um projeto específico, crucial para as necessidades da EMPRESA, com oportunidades de mudar de equipe e projeto à medida que você e nosso negócio dinâmico crescem e evoluem. Precisamos que nossos engenheiros sejam versáteis, demonstrem qualidades de liderança e sejam entusiasmados para enfrentar novos desafios em todo o conjunto de tecnologias, enquanto continuamos a impulsionar a tecnologia. Com sua expertise técnica, você gerenciará prioridades, prazos e entregas de projetos. Você irá projetar, desenvolver, testar, implantar, manter e aprimorar soluções de software. A EMPRESA é uma empresa de engenharia em sua essência. Contratamos pessoas com um amplo conjunto de habilidades técnicas que estejam prontas para enfrentar alguns dos maiores desafios da tecnologia e causar impacto em usuários ao redor do mundo. Na EMPRESA, os engenheiros não apenas revolucionam a pesquisa, mas também trabalham rotineiramente em soluções de escalabilidade e armazenamento, aplicativos de larga escala e plataformas totalmente novas para desenvolvedores em todo o mundo. Da publicidade online ao navegador, do sistema operacional móvel à plataforma de vídeos, do social ao local, os engenheiros da EMPRESA estão mudando o mundo, uma conquista tecnológica de cada vez. Responsabilidades: Escrever código para desenvolvimento de produto ou sistema. Participar ou liderar revisões de design com colegas e stakeholders para decidir entre as tecnologias disponíveis. Revisar o código desenvolvido por outros desenvolvedores e fornecer feedback para garantir as melhores práticas (por exemplo, diretrizes de estilo, verificação de código, precisão, testabilidade e eficiência). Contribuir para a documentação existente ou conteúdo educacional e adaptar o conteúdo com base em atualizações de produtos/programas e feedback do usuário. Triagem de problemas de produto ou sistema e depuração/rastreamento/resolução, analisando as fontes dos problemas e o impacto nas operações e qualidade de hardware, rede ou serviço."
fsp_sample_feedback1 = "Analisando seu currículo e a descrição da vaga, te dou uma nota 8. Pontos Positivos: Experiência Relevante: Você possui mais de 1 ano de experiência com desenvolvimento de software, incluindo experiência com linguagens de programação mencionadas na descrição da vaga (C, Python) e experiência em projetos relevantes como desenvolvimento embedded e web. Formação Acadêmica: Sua formação em Engenharia Eletrônica e MBA em Engenharia de Software te proporcionam uma base sólida para a vaga. Habilidades Técnicas: Você demonstra conhecimento em áreas importantes como Git, GitHub, metodologias ágeis e testes de software. Experiência com Linux: Você tem experiência com desenvolvimento em Linux, o que é um diferencial positivo. Comunicação: Sua experiência em customer facing demonstra capacidade de comunicação e interação com clientes, o que é valorizado pela Google. Pontos a Melhorar: Tempo de Carreira: Apesar de ter experiência relevante, você não se encaixa no perfil Early Career por ter mais tempo de experiência do que a vaga pede. A vaga é voltada para recém-formados ou com poucos anos de experiência. Especificar Tecnologias: Seria interessante destacar no currículo suas experiências com tecnologias Google (se houver) como Android, Chrome, YouTube, etc. Portfolio: Um portfólio online com projetos relevantes poderia fortalecer sua candidatura. Recomendações: Ajustar o currículo: Adapte seu currículo para destacar as experiências mais relevantes para a vaga. Criar um portfólio online: Mostre seus projetos e habilidades através de um site ou plataforma online. Buscar vagas similares: Explore outras vagas na Google ou em empresas com perfil similar que valorizem suas habilidades e experiência e não se limitem a Early Career. Você tem um perfil forte e com algumas pequenas adaptações, tem grandes chances de ser contratado pela Google ou por empresas que buscam profissionais com suas qualificações!"

# Example prompt/answer 2
fsp_sample_job2 = "Pessoa Desenvolvedora Linux kernel Pleno Apaixonado por sistemas operacionais, C e Rust? Deseja impulsionar sua carreira? Junte-se à nossa equipe de kernel! A EMPRESA está montando um time de especialistas para garantir o funcionamento ideal de diversas plataformas e arquiteturas em nosso ambiente. Como membro da equipe, você terá a oportunidade de: Otimizar soluções de hardware; Certificar novos hardwares; Depurar problemas de infraestrutura relacionados ao kernel; Integrar drivers de terceiros. Buscamos profissionais em início de carreira ou recém-formados com interesse em desenvolvimento de kernel em áreas como: Atualizações de segurança; Patches e testes. Suas responsabilidades incluirão: Aprofundar o conhecimento do kernel Linux e sua distribuição com mentores experientes; Colaborar com equipes internas e externas de hardware; Resolver problemas de kernel identificados por clientes e testes; Aprimorar ferramentas e automação para garantir a qualidade do kernel; Enviar, revisar e aplicar patches do kernel; Manter-se atualizado sobre os avanços do kernel; Escrever código C de alta qualidade; Trabalhar com Git e outras ferramentas em ambiente open source. Qualificações essenciais: Graduação em Ciência da Computação, Matemática, Engenharia ou áreas afins (ou experiência equivalente); Pelo menos 3 anos de experiência em desenvolvimento e integração de kernel Linux em ambientes complexos; Conhecimento em sistemas operacionais, arquitetura de computadores e programação de sistemas; Domínio de kernel Linux, hipervisores e sistemas operacionais open source; Familiaridade com distribuições Linux (Ubuntu, Red Hat, SUSE etc.); Habilidades de colaboração e comunicação; Paixão pelo open source; Experiência em depuração de kernel; Experiência em teste de sistemas e conhecimento de API do espaço do usuário; Proficiência em C, Python e Bash. Diferenciais: Conhecimento em virtualização; Experiência com drivers de dispositivos e Board Support Package (BSP); Conhecimento em Rust; Conhecimento em Assembly para arquiteturas x86; Experiência com High-Performance Computing; Experiência com Real-Time Operating System (RTOS); Experiência com UEFI, ACPI, PCIe, SATA, NVMe e outras especificações de servidores. Ambiente de trabalho: Na EMPRESA, valorizamos a autonomia, responsabilidade e a multidisciplinaridade. Somos ágeis, eficientes e buscamos entregar valor constantemente. Trabalhamos em ritmo acelerado, mas com equilíbrio e qualidade. Junte-se a nós e faça parte da construção de um ambiente tecnológico de ponta!"
fsp_sample_feedback2 = "Sua nota para a vaga de Pessoa Desenvolvedora Linux Kernel Pleno é 6. Pontos Positivos: Experiência com Linux: Você possui experiência com desenvolvimento em Linux, o que é fundamental para a vaga. Conhecimento de C: A vaga exige domínio em C e você o lista como uma de suas principais habilidades. Experiência com desenvolvimento embedded: Sua experiência com desenvolvimento embedded pode ser útil para o trabalho com kernel Linux. Pontos a Melhorar: Tempo de experiência: A vaga pede pelo menos 3 anos de experiência com desenvolvimento e integração de kernel Linux em ambientes de produção, o que você aparenta não ter no momento. Conhecimento específico de kernel Linux: A descrição da vaga enfatiza a necessidade de conhecimento profundo em áreas como kernel Linux, hipervisores e sistemas operacionais open-source. Seu currículo não demonstra esse conhecimento específico. Experiência com ferramentas e tecnologias: A vaga menciona Python, Bash scripting, virtualização, drivers de dispositivos, BSP, Rust e Assembly como diferenciais. Seria interessante adquirir experiência nessas áreas para fortalecer sua candidatura. Recomendações: Especializar-se em desenvolvimento de Kernel Linux: Busque cursos, certificações e projetos que te permitam aprofundar seus conhecimentos em kernel Linux e áreas relacionadas. Desenvolver projetos com tecnologias chave: Crie projetos que utilizem Python, Bash scripting e outras tecnologias mencionadas na vaga para ganhar experiência prática. Contribuir para projetos open-source: Participar de projetos open-source relacionados a kernel Linux pode te dar experiência valiosa e aumentar sua visibilidade na comunidade. Adaptar o currículo: Ajuste seu currículo para destacar as habilidades e experiências mais relevantes para a vaga de Kernel Linux, incluindo palavras-chave da descrição. Com foco e dedicação, você pode adquirir a experiência e os conhecimentos necessários para se tornar um forte candidato para essa vaga no futuro."

# Coletando dados do usuário

Currículo (CV)

In [4]:
cv = input("Cole aqui o seu currículo: ")

Cole aqui o seu currículo: Perfil Sou Engenheiro Eletrônico com MBA em Engenharia de Software e possuo experiência no desenvolvimento de aplicações industriais e web. Minhas principais habilidades estão relacionadas ao desenvolvimento de software em geral, seguindo práticas ágeis e de engenharia de software. Habilidades Programação: C, Python Engenharia de Software: Git, GitHub, Metodologia Ágil, Unity/MinUnit (testes), Linux Idiomas: Português (nativo), Inglês (fluente), Espanhol (avançado), Francês (intermediário) Habilidades Interpessoais: Aprendizado contínuo, comunicação com o cliente Formação MBA em Engenharia de Software - 2022 EMPRESA - São Paulo, SP Engenharia Eletrônica - 2015 EMPRESA - São Bernardo do Campo, SP Engenharia Elétrica - 2013 EMPRESA - Birmingham, AL (EUA) Intercâmbio Acadêmico de um ano pelo Programa Ciências sem Fronteiras, patrocinado pela CAPES Experiência Profissional Engenheiro de Aplicações de Campo - Fevereiro de 2022 - Presente EMPRESA - São Paulo, SP Su

Dados da vaga objetivo

In [5]:
job_title = input("Título/Cargo da vaga de trabalho: ")
job_description = input("Descrição da vaga de trabalho: ")

Título/Cargo da vaga de trabalho: Pessoa Desenvolvedora Linux kernel Pleno
Descrição da vaga de trabalho: Apaixonado por sistemas operacionais, C e Rust? Deseja impulsionar sua carreira? Junte-se à nossa equipe de kernel! A EMPRESA está montando um time de especialistas para garantir o funcionamento ideal de diversas plataformas e arquiteturas em nosso ambiente. Como membro da equipe, você terá a oportunidade de: Otimizar soluções de hardware; Certificar novos hardwares; Depurar problemas de infraestrutura relacionados ao kernel; Integrar drivers de terceiros. Buscamos profissionais em início de carreira ou recém-formados com interesse em desenvolvimento de kernel em áreas como: Atualizações de segurança; Patches e testes. Suas responsabilidades incluirão: Aprofundar o conhecimento do kernel Linux e sua distribuição com mentores experientes; Colaborar com equipes internas e externas de hardware; Resolver problemas de kernel identificados por clientes e testes; Aprimorar ferramentas e a

# Montando o Prompt

In [6]:
fsp_prompt = f"CV: {fsp_sample_cv}\n\nJob description: {fsp_sample_job1}\n\nFeedback: {fsp_sample_feedback1}\n\nJob description: {fsp_sample_job2}\n\nFeedback: {fsp_sample_feedback2}"

prompt = f"CV: {cv}\n\nJob description: {job_title}\n{job_description}"

full_prompt = f"{system_instructions}\n\n{fsp_prompt}\n\n{prompt}\n\nFeedback: "

response = model.generate_content(full_prompt)

# Mostrando a Nota de compatibilidade do currículo (CV) com a vaga, e os pontos positivos e negativos

In [9]:
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

display(to_markdown(response.text))

> Sua nota para a vaga de Pessoa Desenvolvedora Linux Kernel Pleno é **7**.
> 
> **Pontos Positivos:**
> 
> * **Experiência com Linux Embarcado:** Você possui experiência relevante com desenvolvimento em Linux Embarcado, incluindo personalização com Yocto, o que demonstra familiaridade com o sistema operacional e suas camadas. 
> * **Conhecimento de C:** A vaga exige domínio em C e você o lista como uma de suas principais habilidades, tendo experiência em desenvolvimento C para Linux Embarcado.
> * **Experiência com desenvolvimento embedded:** Sua experiência com desenvolvimento embedded, especialmente para microcontroladores, demonstra capacidade de lidar com hardware e software de baixo nível, o que pode ser útil para o trabalho com kernel Linux.
> * **Trabalho com Git:** Você demonstra familiaridade com Git e GitHub para controle de versão, o que é essencial para o desenvolvimento de kernel Linux.
> 
> **Pontos a Melhorar:**
> 
> * **Experiência específica com kernel Linux:** A descrição da vaga enfatiza a necessidade de pelo menos 3 anos de experiência profunda em desenvolvimento e integração de kernel Linux em ambientes de produção, com conhecimento em áreas como hipervisores e sistemas operacionais open-source.  Embora você tenha experiência com Linux Embarcado, seu currículo não explicita experiência com desenvolvimento de kernel em si.
> * **Conhecimento específico de kernel:**  A vaga menciona áreas como patches de kernel, testes de kernel, depuração de kernel e conhecimento de API do espaço do usuário. Seria importante adquirir experiência prática nessas áreas para se candidatar à vaga. 
> * **Diferenciais:** A vaga lista diversas tecnologias como diferenciais, incluindo Python, Bash scripting, virtualização, drivers de dispositivos, BSP, Rust e Assembly. Aprimorar suas habilidades nessas áreas aumentaria suas chances.
> 
> **Recomendações:**
> 
> * **Adquirir experiência com desenvolvimento de kernel Linux:** Busque cursos online, tutoriais, livros e projetos práticos que te permitam desenvolver e integrar código diretamente no kernel Linux. Experimente compilar seu próprio kernel, explorar a documentação oficial do kernel e familiarize-se com ferramentas como `gdb` para depuração. 
> * **Contribuir para projetos open-source:** Participar de projetos open-source relacionados a kernel Linux, mesmo que com pequenas contribuições, proporcionará experiência valiosa, conhecimento prático e visibilidade na comunidade.
> * **Desenvolver projetos com tecnologias chave:** Crie projetos pessoais que utilizem Python, Bash scripting e outras tecnologias mencionadas na vaga como diferenciais para ganhar experiência prática.
> * **Adaptar o currículo:**  Ajuste seu currículo para destacar as habilidades e experiências mais relevantes para a vaga de Kernel Linux, incluindo palavras-chave da descrição da vaga e exemplos concretos de projetos relacionados a kernel.
> 
> Você possui um bom ponto de partida com sua experiência em Linux Embarcado e desenvolvimento C. Com foco e dedicação em adquirir experiência específica com kernel Linux, você pode se tornar um candidato competitivo para essa vaga. 
